In [65]:
# Load JSON to pandas
import pandas as pd
pd.read_json ('PDGS Test  - 1. PDGS case - test3.json')

import json
import pandas as pd
with open('PDGS Test  - 1. PDGS case - test3.json', "r") as json_file:
    f=json_file.read()
    cost1 = json.loads(f)
with open('PDGS Test  - 6. Process and export from a floating vessel - plan 1.json', "r") as json_file:
    f=json_file.read()
    cost2 = json.loads(f)

In [10]:
# finds max depth of each cost 
def depth(x):
    if type(x) is dict and x:
        return 1 + max(depth(x[a]) for a in x)
    if type(x) is list and x:
        return 1 + max(depth(a) for a in x)
    return 0
depth(cost1), depth(cost2)

(17, 17)

In [62]:
from pandas.io.json import json_normalize
df2 = json_normalize(cost1)
df2

,name,purpose,path,properties,description,amount,children,min.value,min.unit,max.value,max.unit
0,CAPEX PDGS Test,None,/capex,[],None,None,"[{'name': '1. PDGS case', 'purpose': None, 'pa...",2.396332e+09,$,2.907662e+09,$


In [ ]:
def nodes_from_tree(tree, parent):
    r = [parent]
    for child in tree.get_children(parent):
        r.extend(nodes_from_tree(tree, child))
    return r

In [59]:
def flatten_json(nested_json):
    """
        Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {} 
    out_l=[]
    def flatten(x, name=''):
        nonlocal out
        if name == "children": out = {}
        if type(x) is dict:
            for a in x:
#                 flatten(x[a], name + a + '_')
                flatten(x[a], a)
#                 if a=="children": flatten(x[a], a)
#                 out[a]=x[a]
#             out_l.append(out)
            
        elif type(x) is list:
            i = 0
            if out != {}: 
                out_l.append(out)
            for a in x:
#                 flatten(a, name + str(i) + '_')
                flatten(a, name)
                i += 1
            
        else:
            out[name] = x
#             out_l.append(out)

    flatten(nested_json)
    return out_l
df=pd.DataFrame(flatten_json(cost1))
df

,name,purpose,path,description,value,unit,amount,properties,min,max
0,CAPEX PDGS Test,None,/capex,None,2.907662e+09,$,NaN,NaN,NaN,NaN
1,1. PDGS case,None,/capex/1. PDGS case,None,2.446674e+09,$,NaN,NaN,NaN,NaN
2,CS jumper,None,/capex/1. PDGS case/CS jumper,None,4.063269e+05,$,NaN,NaN,NaN,NaN
3,CS jumper (SP 1),None,/capex/1. PDGS case/CS jumper/CS jumper (SP 1),"8 in nom OD, 15.88 mm WT",1.637093e+04,$,NaN,Count: 1,NaN,NaN
4,Generic,None,/capex/1. PDGS case/CS jumper/CS jumper (SP 1)...,None,1.637093e+04,$,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2134,Vessel owner 1,None,/capex/test3/csv2/Installation-Flying Leeds/Tr...,None,4.282800e+05,$,NaN,NaN,NaN,NaN
2135,None,Total cost vessel + spread,/capex/test3/csv2/Installation-Flying Leeds/Tr...,None,4.282800e+05,$,NaN,NaN,NaN,NaN
2136,Demobilisation,None,/capex/test3/csv2/Installation-Flying Leeds/De...,None,1.200000e+05,$,NaN,Duration: 1 day,NaN,NaN
2137,Vessel owner 1,None,/capex/test3/csv2/Installation-Flying Leeds/De...,None,1.200000e+05,$,NaN,NaN,NaN,NaN


In [68]:
with open('test.json', "r") as json_file:
    f=json_file.read()
    test = json.loads(f)
out={}
out_l=[]
def removeChildren(d):
    for x in d:
        if type(x) is dict:
            for a in x:
                if a=="children": 
                    removeChildren(x[a])
                out[a]=x[a]
            out_l.append(out)

        elif type(x) is list:
            for a in x:
                removeChildren(a)
    return out_l

In [73]:
for a in test[0]:
    if a=="children": 
        print(test[0][a])

[{'name': '1. PDGS case', 'purpose': None, 'path': '/capex/1. PDGS case', 'properties': [], 'description': None, 'min': {'value': 1948447094.215, 'unit': '$'}, 'max': {'value': 2446674060.9483, 'unit': '$'}, 'amount': None, 'children': [{'name': 'CS jumper', 'purpose': None, 'path': '/capex/1. PDGS case/CS jumper', 'properties': [], 'description': None, 'min': {'value': 406326.8603, 'unit': '$'}, 'max': {'value': 406326.8603, 'unit': '$'}, 'amount': None, 'children': [{'name': 'CS jumper (SP 1)', 'purpose': None, 'path': '/capex/1. PDGS case/CS jumper/CS jumper (SP 1)', 'properties': ['Length: 44.93 m', 'Weight: 5,456.98 kg', 'Count: 1'], 'description': '8 in nom OD, 15.88 mm WT', 'min': {'value': 16370.9286, 'unit': '$'}, 'max': {'value': 16370.9286, 'unit': '$'}, 'amount': None, 'children': [{'name': 'Generic', 'purpose': None, 'path': '/capex/1. PDGS case/CS jumper/CS jumper (SP 1)/Generic', 'properties': [], 'description': None, 'min': None, 'max': None, 'amount': {'value': 16370.9

In [74]:
#level 1
for index, i in enumerate(test):
    print(index, i["name"])
    level = index
#     name=i["name"]
    l=0
    while "childern" in i:
        l+=1
        path="children"*l
        name = i["name"]

0 CAPEX PDGS Test 


In [76]:
global leafArray
leafArray = []


def parseTree(obj):
    if len(obj["children"]) == 0:
        leafArray.append(obj)
    else:
        for child in obj["children"]:
            parseTree(child)
for line in test:
    parseTree(test) 



TypeError: list indices must be integers or slices, not str

In [69]:
removeChildren(test)

[{'name': 'Bend',
  'purpose': None,
  'path': '/capex/1. PDGS case/CS jumper/CS jumper (SP 1)/Components/Bend',
  'properties': ['Length: 4.42 m', 'Weight: 444.75 kg', 'Count: 3'],
  'description': '10in Nom OD, 15.88mm WT',
  'min': {'value': 0, 'unit': '$'},
  'max': {'value': 0, 'unit': '$'},
  'amount': None,
  'children': [{'name': '1. PDGS case',
    'purpose': None,
    'path': '/capex/1. PDGS case',
    'properties': [],
    'description': None,
    'min': {'value': 1948447094.215, 'unit': '$'},
    'max': {'value': 2446674060.9483, 'unit': '$'},
    'amount': None,
    'children': [{'name': 'CS jumper',
      'purpose': None,
      'path': '/capex/1. PDGS case/CS jumper',
      'properties': [],
      'description': None,
      'min': {'value': 406326.8603, 'unit': '$'},
      'max': {'value': 406326.8603, 'unit': '$'},
      'amount': None,
      'children': [{'name': 'CS jumper (SP 1)',
        'purpose': None,
        'path': '/capex/1. PDGS case/CS jumper/CS jumper (SP 1)